In [6]:
import fastjsonschema
import jsonschema
import json
from types import FunctionType
from typing import Callable, Set, Dict
from inspect import getmembers, isfunction
from warnings import warn
from uuid import uuid4, UUID
from functools import partial


class JSONObject:
    idKey: str = "@id"
    classKey: str = "@type"
    classValue: Callable = lambda obj: type(obj).__name__
    schema: Dict = dict()
    __funcs__: Dict = dict()
    refKeys: Set = set()
    refMarkup: Dict = {"graphRef": True, "@type": lambda obj: type(obj).__name__}
    setDefault: bool = False #TODO
    
    
    @staticmethod
    def validate(data):
        return None
    
    @staticmethod
    def JSONDefault(obj):
        if isinstance(obj, UUID):
            return str( obj )
        if isinstance(obj, JSONObject):
            return {"@id": str( getattr(obj, '@id') )}
                
        return json.JSONEncoder.default(obj)
    
    
    def __new__(cls, **kwargs):
        cls.validate = staticmethod( fastjsonschema.compile(cls.schema) )
        
        def JSONDefault(obj):
            if isinstance(obj, UUID):
                return str( obj )
            if isinstance(obj, JSONObject):
                rep = {cls.idKey: str( getattr(obj, cls.idKey) )}
                
                for k in cls.refKeys:
                    rep[k] = obj[k]
                    
                for k, v in cls.refMarkup.items():
                    if isfunction(v):
                        rep[k] = v(obj)
                    else:
                        rep[k] = v
                        
                return rep

            return json.JSONEncoder.default(obj)
        
        cls.JSONDefault = staticmethod( JSONDefault )
        
        return super().__new__(cls)
    
    
    def __init__(self, **kwargs):
        # Init an id
        if (self.idKey in kwargs):
            warn("JSONGraph class may overwrite your manual id.")
        else:
            setattr(self, self.idKey, uuid4())
        # Set the class
#         if ("objectClass")
            
        # Init required vals
        ## Issue if "required" keys override keys checked in setattr
        if "required" in self.schema:
            initial_vals = {k: kwargs[k] for k in self.schema["required"]}

            self.validate(initial_vals)
            self.__dict__.update(initial_vals)
        
        # Iterate to enforce checks on keys
        for k, v in kwargs.items():
            setattr(self, k, v)
            
            
    def json(self):
        return json.dumps(self.__dict__, indent=2, default=self.JSONDefault)
        
        
    def __str__(self):
        return self.json()
    
    
    def __setattr__(self, key, val):
        if key == "schema":
            raise KeyError("Cannot Reset Schema")
            
        if isfunction(val):
            if key not in self.__dict__:
                warn("Functions will not be serialized. Add a callable JSONObject instead.")
            else:
                raise ValueError(f"{key} is already defined as an attribute and cannot be " \
                                 f"redefined as a function")
        
        
        self.validate( {**self.__dict__, key: val} )
        
        self.__dict__.update( {key: val} )
        
        
    def __getattr__(self, key):
        if key in self.__funcs__:
            return self.__funcs__[key]
        
        try:
            return self.__dict__[key]
        finally:
            raise AttributeError(f"{key} not a member of {self}")
    
    
class JSONGraph:
    bannedIds: Callable = lambda x: False
    graph: Dict[str, Set] = dict()
    
        
        
    
    
joe = JSONObject(hello="world")
bob = JSONObject(friend=joe)
joe.friend = bob

print(joe)
print(bob)

{
  "@id": "61435428-9a2c-437b-b4b9-3e8188d7e837",
  "hello": "world",
  "friend": {
    "@id": "25c5924d-6a3d-4be3-901e-321c14c5f31f",
    "graphRef": true,
    "objectClass": "JSONObject"
  }
}
{
  "@id": "25c5924d-6a3d-4be3-901e-321c14c5f31f",
  "friend": {
    "@id": "61435428-9a2c-437b-b4b9-3e8188d7e837",
    "graphRef": true,
    "objectClass": "JSONObject"
  }
}


In [70]:
JSONObject.__name__

'JSONObject'

In [65]:
type(joe).__name__

'JSONObject'

In [54]:
getmembers(joe)

[('@id', UUID('eafdd825-0d2f-42ba-bb56-4caf90a0c3d1')),
 ('__class__', __main__.JSONObject),
 ('__delattr__',
  <method-wrapper '__delattr__' of JSONObject object at 0x7f5481efa400>),
 ('__dict__',
  {'@id': UUID('eafdd825-0d2f-42ba-bb56-4caf90a0c3d1'), 'hello': 'world'}),
 ('__dir__', <function JSONObject.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of JSONObject object at 0x7f5481efa400>),
 ('__format__', <function JSONObject.__format__(format_spec, /)>),
 ('__funcs__', {}),
 ('__ge__', <method-wrapper '__ge__' of JSONObject object at 0x7f5481efa400>),
 ('__getattr__',
  <bound method JSONObject.__getattr__ of <__main__.JSONObject object at 0x7f5481efa400>>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of JSONObject object at 0x7f5481efa400>),
 ('__gt__', <method-wrapper '__gt__' of JSONObject object at 0x7f5481efa400>),
 ('__hash__',
  <method-wrapper '__hash__' of JSONObject object at 0x7f5481efa400>),
 ('__init__',
  <bound method JSONObject

In [55]:
class Specific(JSONObject):
    schema = {"type": "object", "properties": {"thing": {"type": "string"}}}
    
spec = Specific(thing="5")

print(spec)
print(spec.thing)

spec.other = 5

print(spec)

try:
    spec.schema = {}
    spec.thing = 5
except (KeyError, fastjsonschema.JsonSchemaValueException):
    print("int value successfully caught")
    
print(spec)
print(spec.schema)

{
  "@id": "e9ca5595-519e-4e1c-96bd-69da5a26ce5b",
  "thing": "5"
}
5
{
  "@id": "e9ca5595-519e-4e1c-96bd-69da5a26ce5b",
  "thing": "5",
  "other": 5
}
int value successfully caught
{
  "@id": "e9ca5595-519e-4e1c-96bd-69da5a26ce5b",
  "thing": "5",
  "other": 5
}
{'type': 'object', 'properties': {'thing': {'type': 'string'}}}


In [57]:
class Specific(JSONObject):
    schema = {"type": "object", "properties": {"thing": {"type": "string"}}}
    
spec = Specific(thing=foo)

print(spec)
print(spec.thing)

spec.other = 5

print(spec)

try:
    spec.schema = {}
    spec.thing = 5
except (KeyError, fastjsonschema.JsonSchemaValueException):
    print("int value successfully caught")
    
print(spec)
print(spec.schema)

<ipython-input-52-38f858484d8e>:77: UserWarning: Functions will not be serialized. Add a callable JSONObject instead.
  warn("Functions will not be serialized. Add a callable JSONObject instead.")


JsonSchemaValueException: data.thing must be string

In [4]:
getmembers(spec)

[('@id', UUID('7e2b3fe3-61be-428f-8df4-5e4f4d453239')),
 ('__class__', __main__.Specific),
 ('__delattr__',
  <method-wrapper '__delattr__' of Specific object at 0x7f50983603a0>),
 ('__dict__',
  {'@id': UUID('7e2b3fe3-61be-428f-8df4-5e4f4d453239'),
   'thing': '5',
   'other': 5}),
 ('__dir__', <function Specific.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of Specific object at 0x7f50983603a0>),
 ('__format__', <function Specific.__format__(format_spec, /)>),
 ('__funcs__', {}),
 ('__ge__', <method-wrapper '__ge__' of Specific object at 0x7f50983603a0>),
 ('__getattr__',
  <bound method JSONObject.__getattr__ of <__main__.Specific object at 0x7f50983603a0>>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of Specific object at 0x7f50983603a0>),
 ('__gt__', <method-wrapper '__gt__' of Specific object at 0x7f50983603a0>),
 ('__hash__',
  <method-wrapper '__hash__' of Specific object at 0x7f50983603a0>),
 ('__init__',
  <bound method JSONObject.__ini

In [5]:
def foo():
    print("foo")
    
spec.foo = foo

getmembers(spec)

<ipython-input-2-0355c3d9737d>:62: UserWarning: Functions will not be serialized. Add a callable JSONObject instead.
  warn("Functions will not be serialized. Add a callable JSONObject instead.")


[('@id', UUID('7e2b3fe3-61be-428f-8df4-5e4f4d453239')),
 ('__class__', __main__.Specific),
 ('__delattr__',
  <method-wrapper '__delattr__' of Specific object at 0x7f50983603a0>),
 ('__dict__',
  {'@id': UUID('7e2b3fe3-61be-428f-8df4-5e4f4d453239'),
   'thing': '5',
   'other': 5}),
 ('__dir__', <function Specific.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of Specific object at 0x7f50983603a0>),
 ('__format__', <function Specific.__format__(format_spec, /)>),
 ('__funcs__', {'foo': <function __main__.foo()>}),
 ('__ge__', <method-wrapper '__ge__' of Specific object at 0x7f50983603a0>),
 ('__getattr__',
  <bound method JSONObject.__getattr__ of <__main__.Specific object at 0x7f50983603a0>>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of Specific object at 0x7f50983603a0>),
 ('__gt__', <method-wrapper '__gt__' of Specific object at 0x7f50983603a0>),
 ('__hash__',
  <method-wrapper '__hash__' of Specific object at 0x7f50983603a0>),
 ('__init__',


In [6]:
import marshal

marshal.dumps( foo.__code__ )

b'\xe3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00C\x00\x00\x00s\x0c\x00\x00\x00t\x00d\x01\x83\x01\x01\x00d\x00S\x00)\x02N\xda\x03foo)\x01\xda\x05print\xa9\x00r\x03\x00\x00\x00r\x03\x00\x00\x00\xfa\x1e<ipython-input-5-dd64f9b99ec5>r\x01\x00\x00\x00\x01\x00\x00\x00s\x02\x00\x00\x00\x00\x01'

In [7]:
getmembers(foo)

[('__annotations__', {}),
 ('__call__',
  <method-wrapper '__call__' of function object at 0x7f50983129d0>),
 ('__class__', function),
 ('__closure__', None),
 ('__code__',
  <code object foo at 0x7f5098327b30, file "<ipython-input-5-dd64f9b99ec5>", line 1>),
 ('__defaults__', None),
 ('__delattr__',
  <method-wrapper '__delattr__' of function object at 0x7f50983129d0>),
 ('__dict__', {}),
 ('__dir__', <function function.__dir__()>),
 ('__doc__', None),
 ('__eq__', <method-wrapper '__eq__' of function object at 0x7f50983129d0>),
 ('__format__', <function function.__format__(format_spec, /)>),
 ('__ge__', <method-wrapper '__ge__' of function object at 0x7f50983129d0>),
 ('__get__', <method-wrapper '__get__' of function object at 0x7f50983129d0>),
 ('__getattribute__',
  <method-wrapper '__getattribute__' of function object at 0x7f50983129d0>),
 ('__globals__',
  {'__name__': '__main__',
   '__doc__': 'Automatically created module for IPython interactive environment',
   '__package__': N